In [5]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

IOStream.flush timed out
You should consider upgrading via the '/opt/homebrew/anaconda3/envs/lotus/bin/python -m pip install --upgrade pip' command.


In [1]:
from pyspark.sql import SparkSession
import os

# Initialize Spark Session with the custom spark runtime jar that contains the spark action for embedding
spark = SparkSession.builder \
    .appName("OpenHouse-Example") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0-SNAPSHOT") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", f"{os.getcwd()}/warehouse") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", f"{os.getcwd()}/local/warehouse") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/lejiang/sources/spark-3.5.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/lejiang/.ivy2/cache
The jars for the packages stored in: /Users/lejiang/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c4b5f016-c5ce-40ec-8f52-921f210aaedb;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0-SNAPSHOT in local-m2-cache
:: resolution report :: resolve 56ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0-SNAPSHOT from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	-----------------------------------------------------------------

In [2]:
from openhouse_connector import SparkConnector, OpenHouse

# Create `employees` table in Spark
spark.sql("CREATE DATABASE IF NOT EXISTS db")
spark.sql("DROP TABLE IF EXISTS db.employees")
spark.sql("CREATE TABLE IF NOT EXISTS db.employees (id INT, name STRING, age INT) USING iceberg")

# Initialize OpenHouse with SparkConnector
openhouse = OpenHouse(SparkConnector(spark))

/opt/homebrew/anaconda3/envs/lotus/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/anaconda3/envs/lotus/lib/python3.10/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
2025-01-16 16:57:06,122 - INFO - Use pytorch device: mps


In [3]:
import pandas as pd

# Create sample data as a pandas DataFrame
data = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "name": ["Emma", "Michael", "Sophia", "James", "Olivia"],
    "age": [27, 33, 29, 45, 31]
})

data2 = pd.DataFrame({
    "id": [6, 7, 8, 9, 10],
    "name": ["Lisa", "Omar", "Nina", "Peter", "Rachel"],
    "age": [31, 45, 28, 39, 33]
})

In [6]:
# Write to table and search the content with semantic query
table = openhouse.table("db.employees")
table.write(data, index_cols=["name"])
employees = table.head(5)
outdf = employees.sem_search("name", "Who's emma?", K=2, n_rerank=4)
print(outdf)

2025-01-16 17:02:02,549 - INFO - Error while sending or receiving.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/lotus/lib/python3.10/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
BrokenPipeError: [Errno 32] Broken pipe
2025-01-16 17:02:02,609 - INFO - Closing down clientserver connection
2025-01-16 17:02:02,610 - INFO - Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/lotus/lib/python3.10/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
BrokenPipeError: [Errno 32] Broken pipe

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/lotus/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/homebrew/anaconda3/

   id    name  age
0   1    Emma   27
4   5  Olivia   31


In [ ]:
# Clean up
spark.stop()